In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.autograd import Variable

In [2]:
# Load preprocessed raw data

raw_df = pd.read_csv('input_data_2021-12.csv', names = ['ud','ma','volume','behav'])
raw_df = raw_df.astype('float32')

In [3]:
raw_df

,ud,ma,volume,behav
0,-25.280001,197.162994,3.077591,0.425532
1,28.860001,180.410995,3.026688,0.568965
2,13.400000,163.350006,2.462048,0.543478
3,-34.180000,147.085007,2.963388,0.593750
4,24.379999,137.246506,2.557441,0.529412
...,...,...,...,...
44015,-33.099998,-61.709000,2.811403,0.230769
44016,-2.690000,-72.959503,2.919086,0.250000
44017,-1.650000,-86.261002,3.485018,0.125000
44018,-62.020000,-99.489502,3.991151,0.551724


In [5]:
# Min-Max Scaling for each features
# Convert "behav" column's scale from 0~1 to -1~1 (This is target feature)
# Further scale conversion of "behav" column is for the prevention of the skewed training of the model

df = raw_df.copy()
df = (df - raw_df.min(axis=0)) / (raw_df.max(axis=0)-raw_df.min(axis=0))
df['behav'] = 2*df['behav']-1

In [6]:
df

,ud,ma,volume,behav
0,0.415733,0.828605,0.436649,-0.169453
1,0.596200,0.800685,0.424811,0.122537
2,0.544667,0.772250,0.293500,0.070652
3,0.386067,0.745142,0.410090,0.172991
4,0.581267,0.728744,0.315684,0.042017
...,...,...,...,...
44015,0.389667,0.397152,0.374745,-0.565934
44016,0.491033,0.378401,0.399787,-0.526786
44017,0.494500,0.356232,0.531400,-0.781250
44018,0.293267,0.334184,0.649105,0.087438


In [7]:
# Median value for each columns

threshold = torch.tensor((raw_df[['ud','ma','volume']].median(axis=0)- raw_df[['ud','ma','volume']].min(axis=0))/ (raw_df[['ud','ma','volume']].max(axis=0)-raw_df[['ud','ma','volume']].min(axis=0))).float()
print(threshold)

tensor([0.5000, 0.4996, 0.3838])


In [10]:
# torch.tensor version of df

data_list = torch.tensor(df.values).float()
print(data_list)

tensor([[ 0.4157,  0.8286,  0.4366, -0.1695],
        [ 0.5962,  0.8007,  0.4248,  0.1225],
        [ 0.5447,  0.7722,  0.2935,  0.0707],
        ...,
        [ 0.4945,  0.3562,  0.5314, -0.7812],
        [ 0.2933,  0.3342,  0.6491,  0.0874],
        [ 0.6020,  0.3104,  0.4513,  0.0150]])


# GAN Model 1 (Only linear layers)

Code for this model is only for demonstration.

This model does not include testing process

In [11]:
image_width = 20    # Time horizon for each image (20 minutes)
image_depth = 3     # View each feature as each channel

# Obtain dataset by applying sliding window of size 20 to data_list 
data_set = torch.zeros((data_list.size()[0] - image_width + 1, image_width, data_list.size()[1]))
for i in range(data_list.size()[0] - image_width + 1):
  data_set[i,:,:] = data_list[i:i+image_width,:]
print(data_set.size())

torch.Size([44001, 20, 4])


In [12]:
image_set = data_set[:,:,:3]   # Input features
latent_set = data_set[:,:,3]   # Target feaure

print(image_set.size())
print(latent_set.size())

torch.Size([44001, 20, 3])
torch.Size([44001, 20])


In [8]:
# Choose training and validation data randomly (95% for training, 5% for Validation)
rand_idx = np.random.choice(data_set.size()[0],size=data_set.size()[0], replace=False)
train_idx = rand_idx[:(data_set.size()[0]//20)*19]
test_idx = rand_idx[(data_set.size()[0]//20)*19:]

train_image = image_set[train_idx,:,:]  # Train input features
train_latent = latent_set[train_idx,:]  # Train targer feature
test_image = image_set[test_idx,:,:]    # Validation input features
test_latent = latent_set[test_idx,:]    # Validation target feature

print(train_image.size())
print(train_latent.size())
print(test_image.size())
print(test_latent.size())

torch.Size([41800, 20, 3])
torch.Size([41800, 20])
torch.Size([2201, 20, 3])
torch.Size([2201, 20])


In [11]:
# Put above training and validation data into Dataloader of Pytorch

train_set = TensorDataset(train_image, train_latent)
test_set = TensorDataset(test_image, test_latent)

bs = 16
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=1, shuffle=False)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Generator(nn.Module):
    '''
    4 linear layers with leaky_relu activation for first 3 layers, and tanh activation for last one.
    Tanh activation at last layer restricts the range of output value to be -1 ~ 1
    This is for the range matching with the target feature ("behav" column)
    '''
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 40)
        self.fc2 = nn.Linear(self.fc1.out_features, 60)
        self.fc3 = nn.Linear(self.fc2.out_features, 80)
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)
    
    # forward method
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        return torch.tanh(self.fc4(x))
    
    
class Discriminator(nn.Module):
    '''
    4 linear layers with leaky_relu activation and dropout for first 3 layers, and sigmoid activation for last one.
    Sigmoid activation at last layer restricts the range of output value to be 0 ~ 1
    Discriminator generally returns the probability of Generator being fake, so it returns pseudo-probabilistic values
    '''    
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 40)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)
    
    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))
    

z_dim = image_width

G = Generator(g_input_dim = z_dim, g_output_dim = (image_width)*(image_depth)).to(device)
D = Discriminator((image_width)*(image_depth)).to(device)

G_optimizer = optim.Adam(G.parameters(), lr = 0.0000035)
D_optimizer = optim.Adam(D.parameters(), lr = 0.0000035)


# Training process
n_epoch = 100
for epoch in range(n_epoch):           
    D_losses, G_losses = [], []
    for x, y in train_loader:
        D.zero_grad()
        x_real = x.view(-1,(image_width)*(image_depth)).to(device)
        D_real_loss = torch.mean(torch.log(D(x_real)))
        Z = y.view(-1,image_width).to(device)
        D_fake_loss = torch.mean(torch.log(1-D(G(Z))))
        D_loss = -(D_real_loss + D_fake_loss)
        D_loss.backward()
        D_optimizer.step()
        D_losses.append(D_loss.data.item())
        
        G.zero_grad()
        Z = y.view(-1,image_width).to(device)
        G_loss = torch.mean(torch.log(1-D(G(Z))))
        G_loss.backward()
        G_optimizer.step()
        G_losses.append(G_loss.data.item())

    D_losses, G_losses = torch.tensor(D_losses), torch.tensor(G_losses) 
    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (epoch, n_epoch, torch.mean(D_losses), torch.mean(G_losses)))   
    

[0/100]: loss_d: 1.381, loss_g: -0.656
[1/100]: loss_d: 1.320, loss_g: -0.668
[2/100]: loss_d: 1.311, loss_g: -0.731
[3/100]: loss_d: 1.379, loss_g: -0.786
[4/100]: loss_d: 1.409, loss_g: -0.732
[5/100]: loss_d: 1.410, loss_g: -0.666
[6/100]: loss_d: 1.403, loss_g: -0.633
[7/100]: loss_d: 1.394, loss_g: -0.630
[8/100]: loss_d: 1.387, loss_g: -0.641
[9/100]: loss_d: 1.387, loss_g: -0.655
[10/100]: loss_d: 1.391, loss_g: -0.660
[11/100]: loss_d: 1.392, loss_g: -0.666
[12/100]: loss_d: 1.388, loss_g: -0.675
[13/100]: loss_d: 1.389, loss_g: -0.678
[14/100]: loss_d: 1.386, loss_g: -0.678
[15/100]: loss_d: 1.385, loss_g: -0.684
[16/100]: loss_d: 1.390, loss_g: -0.689
[17/100]: loss_d: 1.390, loss_g: -0.695
[18/100]: loss_d: 1.385, loss_g: -0.696
[19/100]: loss_d: 1.388, loss_g: -0.694
[20/100]: loss_d: 1.386, loss_g: -0.689
[21/100]: loss_d: 1.389, loss_g: -0.690
[22/100]: loss_d: 1.385, loss_g: -0.700
[23/100]: loss_d: 1.391, loss_g: -0.707
[24/100]: loss_d: 1.387, loss_g: -0.691
[25/100]: 

In [ ]:
# Validation process
# This includes the masked prediction process

loss_list = []
pred_list = torch.zeros(len(test_loader), 1, image_width, image_depth)
orig_list = torch.zeros(len(test_loader), 1, image_width, image_depth)

for iter, (images, latents) in enumerate(test_loader):
  image_num = images.size()[0]
  original = images.clone()
  prediction = torch.zeros(image_num, image_width, image_depth)
  
  
  # Masking process for the last time index (last 1 minute) of the image
  # Masking is done by setting values equal to the median values of the whole data
  mask = torch.ones_like(images, dtype=torch.bool)
  mask[:,-1,:] = 0
  images[mask.logical_not()] = threshold * torch.ones_like(images[mask.logical_not()])
  X = images.clone().requires_grad_(True)
  
  
  objective_list = []
  
  for i in range(100):    
    Z = latents.view(-1,image_width).to(device)
    objective = torch.mean(torch.log(D(X.view(-1,(image_width)*(image_depth)).to(device)))+torch.log(1-D(G(Z))))
    objective.backward()
    objective_list.append(objective)
    X_grad = X.grad.requires_grad_(False)
    X = X.requires_grad_(False)
    projection = torch.clamp(X.data[mask.logical_not()] - 0.05*X_grad[mask.logical_not()], min=0, max=1)
    X.data[mask.logical_not()] = projection
    X.requires_grad_(True)
  prediction = X
  pred_list[iter,:,:,:] = prediction.requires_grad_(False)
  orig_list[iter,:,:,:] = original
  
  # Loss is computed for the last time index (masked index) only
  loss = torch.mean(torch.square(prediction[:,-1,0] - original[:,-1,0]))
  loss_list.append(loss)
  
  print(str(iter)+'th iter done')
  
pred_list = torch.squeeze(pred_list, 1).numpy()
orig_list = torch.squeeze(orig_list, 1).numpy()
loss_list = torch.tensor(loss_list)
print(torch.sqrt(loss_list.mean()))

0th iter done
1th iter done
2th iter done
3th iter done
4th iter done
5th iter done
6th iter done
7th iter done
8th iter done
9th iter done
10th iter done
11th iter done
12th iter done
13th iter done
14th iter done
15th iter done
16th iter done
17th iter done
18th iter done
19th iter done
20th iter done
21th iter done
22th iter done
23th iter done
24th iter done
25th iter done
26th iter done
27th iter done
28th iter done
29th iter done
30th iter done
31th iter done
32th iter done
33th iter done
34th iter done
35th iter done
36th iter done
37th iter done
38th iter done
39th iter done
40th iter done
41th iter done
42th iter done
43th iter done
44th iter done
45th iter done
46th iter done
47th iter done
48th iter done
49th iter done
50th iter done
51th iter done
52th iter done
53th iter done
54th iter done
55th iter done
56th iter done
57th iter done
58th iter done
59th iter done
60th iter done
61th iter done
62th iter done
63th iter done
64th iter done
65th iter done
66th iter done
67th 

In [ ]:
result_df = raw_df[['ud','ma','volume']].copy()
pred = pd.DataFrame(pred_list[:,-1,:], columns=['ud','ma','volume']) * (result_df.max(axis=0)-result_df.min(axis=0)) + result_df.min(axis=0)
orig = pd.DataFrame(orig_list[:,-1,:], columns=['ud','ma','volume']) * (result_df.max(axis=0)-result_df.min(axis=0)) + result_df.min(axis=0)

print(pred)
print(orig)

             ud          ma    volume
0     -2.184845   -7.343445  3.244495
1     31.056152   15.320679  2.050536
2     50.184326   65.647644  2.418587
3     33.258682   99.633026  2.227174
4     10.134796   94.862488  2.716860
...         ...         ...       ...
2196  99.784927   23.788025  2.210980
2197  57.828079  300.000000  2.207002
2198  61.330368   58.672852  3.850240
2199  66.190018   90.189453  2.184935
2200  70.684891   81.120026  3.537884

[2201 rows x 3 columns]
             ud         ma    volume
0     -0.029999  59.796509  1.335180
1     -4.809998 -27.092010  3.636848
2     55.830002 -22.984985  3.948858
3      0.410004 -10.763000  2.507813
4    -34.500000  -6.591003  2.464006
...         ...        ...       ...
2196  -4.070007 -36.807007  3.106523
2197 -17.009995 -90.885010  1.484441
2198 -19.979996  -4.312500  2.381502
2199  16.110016 -32.662994  2.336036
2200  17.580002  68.588013  2.891140

[2201 rows x 3 columns]


In [ ]:
# We can see that predicted values are much right-skewed compared to the original values

print(len(orig[orig['ud']>0]))
print(len(orig[orig['ud']<0]))
print(len(pred[pred['ud']>0]))
print(len(pred[pred['ud']<0]))

1087
1112
2090
111


In [ ]:
orig_big = orig[orig['ud'] > 0]
big_correct = 0
for i in orig_big.index:
  if pred.loc[i,'ud'] > 0:
    big_correct += 1
big_correct_rate = big_correct / len(orig_big.index)
print(big_correct_rate)

0.3581395348837209


In [ ]:
pred_big = pred[pred['ud'] > 40]
big_correct = 0
for i in pred_big.index:
  if orig.loc[i,'ud'] > -40:
    big_correct += 1
big_correct_rate = big_correct / len(pred_big.index)
print(big_correct_rate)

0.7142857142857143


In [ ]:
small = orig[orig['ud'] < 0]
small_correct = 0
for i in small.index:
  if pred.loc[i,'ud'] < 0:
    small_correct += 1
small_correct_rate = small_correct / len(small.index)
print(small_correct_rate)

0.647636039250669


In [ ]:
pred_small = pred[pred['ud'] < -40]
small_correct = 0
for i in pred_small.index:
  if orig.loc[i,'ud'] < 40:
    small_correct += 1
small_correct_rate = small_correct / len(pred_small.index)
print(small_correct_rate)

0.9150943396226415


# GAN Model 2 (Convolutional Layers)

In [14]:
image_width2 = 20    # Time horizon for each image (20 minutes)
image_depth2 = 3     # View each feature as each channel

# Obtain dataset by applying sliding window of size 20 to data_list 
data_set2 = torch.zeros((data_list.size()[0] - image_width2 + 1, image_width2, data_list.size()[1]))
for i in range(data_list.size()[0] - image_width2 + 1):
  data_set2[i,:,:] = data_list[i:i+image_width2,:]
data_set2 = torch.transpose(data_set2, 1, 2)
print(data_set2.size())

torch.Size([44001, 4, 20])


In [15]:
# Second and third dimension are transposed compared to the case of the Linear-layer-only model
# this is because nn.Linear views the third dimension as the channnels
# while nn.Conv1d view the second dimension as the channels

image_set2 = data_set2[:,:3,:]                     # Input features
latent_set2 = torch.unsqueeze(data_set2[:,3,:],1)  # Target feature

print(image_set2.size())
print(latent_set2.size())

torch.Size([44001, 3, 20])
torch.Size([44001, 1, 20])


In [16]:
# Choose training and validation data randomly (80% for training, 10% for validation, 10% for testing)
rand_idx = np.random.choice(data_set2.size()[0],size=data_set2.size()[0], replace=False)
train_idx = rand_idx[:(data_set2.size()[0]//10)*8]
val_idx = rand_idx[(data_set2.size()[0]//10)*8:(data_set2.size()[0]//10)*9]
test_idx = rand_idx[(data_set2.size()[0]//10)*9:]

train_image2 = image_set2[train_idx,:,:]   # Train input features
train_latent2 = latent_set2[train_idx,:]   # Train target feature
val_image2 = image_set2[val_idx,:,:]       # Validation train features
val_latent2 = latent_set2[val_idx,:]       # Validation target feature
test_image2 = image_set2[test_idx,:,:]     # Test train features
test_latent2 = latent_set2[test_idx,:]     # Test target feature

print(train_image2.size())
print(train_latent2.size())
print(val_image2.size())
print(val_latent2.size())
print(test_image2.size())
print(test_latent2.size())

torch.Size([35200, 3, 20])
torch.Size([35200, 1, 20])
torch.Size([4400, 3, 20])
torch.Size([4400, 1, 20])
torch.Size([4401, 3, 20])
torch.Size([4401, 1, 20])


In [13]:
# Put above training, validation, test data into Dataloader of Pytorch

train_set2 = TensorDataset(train_image2, train_latent2)
val_set2 = TensorDataset(val_image2, val_latent2)
test_set2 = TensorDataset(test_image2, test_latent2)

bs2 = 16
train_loader2 = torch.utils.data.DataLoader(dataset=train_set2, batch_size=bs2, shuffle=True)
val_loader2 = torch.utils.data.DataLoader(dataset=val_set2, batch_size=1, shuffle=False)
test_loader2 = torch.utils.data.DataLoader(dataset=test_set2, batch_size=1, shuffle=False)

In [14]:
class Generator(nn.Module):
    '''
    2 Conv1d layers, then 1 Squeeze-and-Excitation layer, then 2 Conv1d layers
    '''
    def __init__(self, latent_dim, image_dim):
        super(Generator, self).__init__()
        lat = latent_dim       
        self.layer1 = nn.Sequential(
            nn.Conv1d(lat, lat*2, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm1d(lat*2),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(lat*2, lat*4, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm1d(lat*4),
            nn.ReLU()
        )
        self.globalAvgPool = nn.AdaptiveAvgPool1d(1)
        self.se = nn.Sequential(
            nn.Linear(lat*4, lat),
            nn.ReLU(),
            nn.Linear(lat, lat*4),
            nn.Sigmoid()
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(lat*4, lat*8, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm1d(lat*8),
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Conv1d(lat*8, image_dim, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm1d(image_dim),
            nn.ReLU()
        )
    
    # forward method
    def forward(self, x): 
        out = self.layer1(x)
        out = self.layer2(out)
        avg_out = torch.squeeze(self.globalAvgPool(out))
        se_out = self.se(avg_out).view(out.size()[0], out.size()[1], 1)
        out = out * se_out.expand_as(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return torch.sigmoid(out)
    
    
class Discriminator(nn.Module):
    '''
    2 Conv1d layers, then 1 Squeeze-and-Excitation layer, then 2 Conv1d layers
    '''
    def __init__(self, image_dim):
        super(Discriminator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(image_dim, image_dim*12, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm1d(image_dim*12),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(image_dim*12, image_dim*8, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm1d(image_dim*8),
            nn.ReLU()
        )
        self.globalAvgPool = nn.AdaptiveAvgPool1d(1)
        self.se = nn.Sequential(
            nn.Linear(image_dim*8, image_dim*2),
            nn.ReLU(),
            nn.Linear(image_dim*2, image_dim*8),
            nn.Sigmoid()
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(image_dim*8, image_dim*4, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm1d(image_dim*4),
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Conv1d(image_dim*4, 1, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm1d(1),
            nn.ReLU()
        )
    
    # forward method
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        avg_out = torch.squeeze(self.globalAvgPool(out))
        se_out = self.se(avg_out).view(out.size()[0], out.size()[1], 1)
        out = out * se_out.expand_as(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return torch.sigmoid(out)
    

z_dim = 1

G = Generator(latent_dim = z_dim, image_dim = image_depth2).to(device)
D = Discriminator(image_dim = image_depth2).to(device)

G_optimizer = optim.Adam(G.parameters(), lr = 0.0000035)
D_optimizer = optim.Adam(D.parameters(), lr = 0.0000035)

In [ ]:
# Training process

n_epoch = 100

# Alternate the optimization process (1 time after 1 time) of Discriminator and Generator
for epoch in range(n_epoch):           
    D_losses, G_losses = [], []
    for x, y in train_loader2:
        D.zero_grad()
        x_real = x.to(device)
        D_real_loss = torch.mean(torch.log(D(x_real)))
        Z = y.to(device)
        D_fake_loss = torch.mean(torch.log(1-D(G(Z))))
        D_loss = -(D_real_loss + D_fake_loss)
        D_loss.backward()
        D_optimizer.step()
        D_losses.append(D_loss.data.item())
        
        G.zero_grad()
        
        Z = y.to(device)
        G_loss = torch.mean(torch.log(1-D(G(Z))))
        G_loss.backward()
        G_optimizer.step()
        G_losses.append(G_loss.data.item())
    D_losses, G_losses = torch.tensor(D_losses), torch.tensor(G_losses)    
    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (epoch, n_epoch, torch.mean(D_losses), torch.mean(G_losses)))

[0/100]: loss_d: 1.473, loss_g: -0.913
[1/100]: loss_d: 1.438, loss_g: -0.881
[2/100]: loss_d: 1.418, loss_g: -0.863
[3/100]: loss_d: 1.407, loss_g: -0.852
[4/100]: loss_d: 1.400, loss_g: -0.844
[5/100]: loss_d: 1.394, loss_g: -0.835
[6/100]: loss_d: 1.390, loss_g: -0.829
[7/100]: loss_d: 1.388, loss_g: -0.825
[8/100]: loss_d: 1.383, loss_g: -0.819
[9/100]: loss_d: 1.377, loss_g: -0.812
[10/100]: loss_d: 1.371, loss_g: -0.805
[11/100]: loss_d: 1.366, loss_g: -0.799
[12/100]: loss_d: 1.362, loss_g: -0.794
[13/100]: loss_d: 1.360, loss_g: -0.791
[14/100]: loss_d: 1.360, loss_g: -0.788
[15/100]: loss_d: 1.359, loss_g: -0.786
[16/100]: loss_d: 1.358, loss_g: -0.783
[17/100]: loss_d: 1.357, loss_g: -0.779
[18/100]: loss_d: 1.351, loss_g: -0.771
[19/100]: loss_d: 1.343, loss_g: -0.761
[20/100]: loss_d: 1.337, loss_g: -0.754
[21/100]: loss_d: 1.334, loss_g: -0.750
[22/100]: loss_d: 1.335, loss_g: -0.750
[23/100]: loss_d: 1.332, loss_g: -0.746
[24/100]: loss_d: 1.327, loss_g: -0.740
[25/100]: 

In [ ]:
# Save the weight values of the trained Generator and Discriminator

torch.save(G.state_dict(), '/content/generator.pt')
torch.save(D.state_dict(), '/content/discriminator.pt')

In [15]:
# Load the weight values of the trained Generator and Disciminator

G = Generator(latent_dim = z_dim, image_dim = image_depth2).to(device)
D = Discriminator(image_dim = image_depth2).to(device)
G.load_state_dict(torch.load('generator.pt',map_location=device))
D.load_state_dict(torch.load('discriminator.pt',map_location=device))
G.eval()
D.eval()
G.requires_grad_(False)
D.requires_grad_(False)

Discriminator(
  (layer1): Sequential(
    (0): Conv1d(3, 36, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv1d(36, 24, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (globalAvgPool): AdaptiveAvgPool1d(output_size=1)
  (se): Sequential(
    (0): Linear(in_features=24, out_features=6, bias=True)
    (1): ReLU()
    (2): Linear(in_features=6, out_features=24, bias=True)
    (3): Sigmoid()
  )
  (layer3): Sequential(
    (0): Conv1d(24, 12, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv1d(12, 1, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): Ba

In [ ]:
# Validation process
# This includes the masked prediction process

loss_list2 = []
pred_list2 = torch.zeros(len(val_loader2), 1, image_depth2, image_width2)
orig_list2 = torch.zeros(len(val_loader2), 1, image_depth2, image_width2)
disc_list2 = torch.zeros(len(val_loader2))

for iter, (images, latents) in enumerate(val_loader2):
  image_num = images.size()[0]
  original = images.clone()
  prediction = torch.zeros(image_num, image_depth2, image_width2)
  
  
  # Masking process for the last time index (last 1 minute) of the image
  # Masking is done by setting values equal to the median values of the whole data
  mask = torch.ones_like(images, dtype=torch.bool)
  mask[:,:,-1] = 0
  images[mask.logical_not()] = threshold*torch.ones_like(images[mask.logical_not()])
  X = images.clone().requires_grad_(True)
  
  
  objective_list = []
  for i in range(100):    
    Z = latents.to(device)
    objective = torch.mean(torch.log(D(X.to(device)))+torch.log(1-D(G(Z))))
    objective.backward()
    objective_list.append(objective)
    X_grad = X.grad.requires_grad_(False)
    X = X.requires_grad_(False)
    projection = torch.clamp(X.data[mask.logical_not()] 
                             - 0.001*X_grad[mask.logical_not()], min=0, max=1)
    X.data[mask.logical_not()] = projection
    X.requires_grad_(True)
  prediction = X
  pred_list2[iter,:,:,:] = prediction.requires_grad_(False)
  orig_list2[iter,:,:,:] = original
  disc_list2[iter] = torch.mean(D(X.to(device)))
  
  # Loss is computed for the last time index (masked index) only
  loss = torch.mean(torch.square(prediction[:,0,-1] - original[:,0,-1]))
  loss_list2.append(loss)
  
  print(str(iter)+'th iter completed')
loss_list2 = torch.tensor(loss_list2)
pred_list2 = torch.squeeze(pred_list2, 1).numpy()
orig_list2 = torch.squeeze(orig_list2, 1).numpy()
print(torch.sqrt(loss_list2.mean()))

0th iter completed
1th iter completed
2th iter completed
3th iter completed
4th iter completed
5th iter completed
6th iter completed
7th iter completed
8th iter completed
9th iter completed
10th iter completed
11th iter completed
12th iter completed
13th iter completed
14th iter completed
15th iter completed
16th iter completed
17th iter completed
18th iter completed
19th iter completed
20th iter completed
21th iter completed
22th iter completed
23th iter completed
24th iter completed
25th iter completed
26th iter completed
27th iter completed
28th iter completed
29th iter completed
30th iter completed
31th iter completed
32th iter completed
33th iter completed
34th iter completed
35th iter completed
36th iter completed
37th iter completed
38th iter completed
39th iter completed
40th iter completed
41th iter completed
42th iter completed
43th iter completed
44th iter completed
45th iter completed
46th iter completed
47th iter completed
48th iter completed
49th iter completed
50th iter 

In [23]:
# Testing process
# First, we find out the anomalous validation data
# and compute similarity score of each test data with each data within this set of anomaly validation data
# If similar enough, then regard that test image as the anomlous one as well
# When compuiting similarity score, we only compare the anomalous locations within the anomaly validation data

# If the average value of the image is larger than 0.6, then we save it as the anomaly validation image 
anomaly_idx = []
for i in range(val_image2.size()[0]):
  if torch.mean(D((torch.unsqueeze(val_image2[i,:,:], 0)).to(device))) > 0.6:
    anomaly_idx.append(i)
val_anomaly = (val_image2[anomaly_idx]).to(device)

# Anomaly locations within each anomaly validation image
anomaly_loc = torch.zeros((val_anomaly.size()[0], 1, 20))
for i in range(val_anomaly.size()[0]):
  anomaly_loc[i] = D((torch.unsqueeze(val_anomaly[i,:,:], 0)).to(device)) > 0.9
anomaly_loc = torch.squeeze(anomaly_loc,1).bool()

# Similarity Score Computation
sim_idx = []
correct_sim_idx = []
total = 0
correct = 0
for i in range(test_image2.size()[0]):
  image = (test_image2[i,0,:]).to(device)
  for j in range(val_anomaly.size()[0]):
    val = val_anomaly[j,0,:]
    
    # Regard the test image is similar to the anomaly validation iamge if
    # mean-squared difference between two images are less than 0.01
    if torch.mean(torch.square(image - val)) < 0.01:
      sim_idx.append((i, j))
      image_loc = anomaly_loc[j,:]
      check_image = image[image_loc]
      check_val = val[image_loc]
      
      # If both images (Declared to be similar according to the model) are
      # bigger than 0.5
      # or smaller than 0.5
      # (allowing marginal error)
      # then we regard this result as the correct one
      # value of 0.5 is chosen because it is the median value for the target feature
      check = (check_image-0.5)*(check_val-0.5)
      total += check.size()[0]
      check_correct = check[check>-0.0001]  # Allow the marginal error within 0.0001
      correct += check_correct.size()[0]
      if check_correct.size()[0] != 0:
          correct_sim_idx.append((i,j))
print(correct / total)
print(correct)
print(total)

0.8333333333333334


In [ ]:
result_df2 = raw_df[['ud','ma','volume']]
pred2 = pd.DataFrame(pred_list2[:,:,-1], columns=['ud','ma','volume']) * (result_df2.max(axis=0)-result_df2.min(axis=0)) + result_df2.min(axis=0)
orig2 = pd.DataFrame(orig_list2[:,:,-1], columns=['ud','ma','volume']) * (result_df2.max(axis=0)-result_df2.min(axis=0)) + result_df2.min(axis=0)

print(pred2)
print(orig2)

              ud          ma    volume
0      33.116638  300.000000  2.552225
1     -26.253067  -57.225845  1.200000
2     -80.757652    7.671753  1.224072
3     -48.028290   26.080078  2.843516
4    -150.000000  300.000000  3.513996
...          ...         ...       ...
4396  -37.459679  300.000000  2.027124
4397  -92.481461 -300.000000  1.308761
4398   21.299286  -47.648544  3.109064
4399  -97.034134 -300.000000  1.200000
4400  -48.062531 -123.967087  1.704117

[4401 rows x 3 columns]
             ud          ma    volume
0     16.709976  -20.219513  2.989818
1      8.720001   15.489502  2.468395
2     37.309998  -58.547516  1.462813
3      8.660019  -43.496002  2.285357
4    -86.849998 -129.641495  3.631495
...         ...         ...       ...
4396   2.419998   -9.723511  2.530834
4397  24.290009    9.428497  2.930780
4398 -57.609993   17.206512  2.843896
4399 -11.050003  -13.787994  1.891968
4400  -1.750000  -38.986511  2.448672

[4401 rows x 3 columns]


In [ ]:
# Skewedness of the distribution of predicted values are much more improved
# compared to the linear-layer-only model

print(len(orig2[orig2['ud']>0]))
print(len(orig2[orig2['ud']<0]))
print(len(pred2[pred2['ud']>0]))
print(len(pred2[pred2['ud']<0]))

2188
2208
1213
3188


In [ ]:
orig_big2 = orig2[orig2['ud'] > 0]
big_correct2 = 0
for i in orig_big2.index:
  if pred2.loc[i,'ud'] > -40:
    big_correct2 += 1
big_correct_rate2 = big_correct2 / len(orig_big2.index)
print(big_correct_rate2)

0.48663101604278075


In [ ]:
pred_big2 = pred2[pred2['ud'] > 40]
big_correct2 = 0
for i in pred_big2.index:
  if orig2.loc[i,'ud'] > 0:
    big_correct2 += 1
big_correct_rate2 = big_correct2 / len(pred_big2.index)
print(big_correct_rate2)

0.7180544105523495


In [ ]:
small2 = orig2[orig2['ud'] < -40]
small_correct2 = 0
for i in small2.index:
  if pred2.loc[i,'ud'] < 40:
    small_correct2 += 1
small_correct_rate2 = small_correct2 / len(small2.index)
print(small_correct_rate2)

0.8748019017432647


In [ ]:
pred_small2 = pred2[pred2['ud'] < 0]
small_correct2 = 0
for i in pred_small2.index:
  if orig2.loc[i,'ud'] < 40:
    small_correct2 += 1
small_correct_rate2 = small_correct2 / len(pred_small2.index)
print(small_correct_rate2)

0.7249058971141782
